<a href="https://colab.research.google.com/github/udlbook/udlbook/blob/main/Trees/SAT_Graph_Coloring_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph coloring

The purpose of this Python notebook is to use investigate using SAT for graph coloring problems.  We'll aim to color a map of the United States in such a way that no two adjacent states have the same color.  We'll both aim to find a valid color labelling and to determine the minimum number of colors for which this is possible.

You should have completed the notebook on SAT constructions before attempting this notebook.

You can save a local copy of this notebook in your Google account and work through it in Colab (recommended) or you can download the notebook and run it locally using Jupyter notebook or similar.

Contact me at iclimbtreesmail@gmail.com if you find any mistakes or have any suggestions.

In [ ]:
# Install relevant packages
!pip install z3-solver
from z3 import *
import numpy as np
from itertools import combinations
import geopandas as gpd
import matplotlib.pyplot as plt

First let's download a map of the United States and form a 49x49 matrix representing the adjacencies of the lower 48 states.  There will be a one in the matrix if two states are adjacent and a zero otherwise.

In [ ]:
# Download map of the US
!wget https://github.com/udlbook/udlbook/raw/refs/heads/main/Trees/cb_2018_us_state_500k.zip
!unzip -o cb_2018_us_state_500k.zip
# Read shape data
# Read the shapefile into a GeoDataFrame
try:
    states = gpd.read_file("cb_2018_us_state_500k.shp")
except FileNotFoundError:
    print("Shapefile not found. Please make sure 'cb_2018_us_state_500k.shp' is in the current directory.")
    exit()

# Reatain just lower 48 states and DC to make it easier to draw
lower48 = states[~states['NAME'].isin(['Alaska', 'Hawaii', 'American Samoa', 'Virgin Islands', \
                                       'Commonwealth of the Northern Mariana Islands', 'Guam', \
                                       'Puerto Rico', 'United States Virgin Islands', 'District of Columbia'])]

# Reset the index to remove gaps
lower48 = lower48.reset_index(drop=True)

In [ ]:
# Get adjacency matrix for remaining 48 states plus DC.

# Get the number of states
num_states = len(lower48)

# Create an empty adjacency matrix
adjacency_matrix = np.zeros((num_states, num_states), dtype=int)

# Iterate through pairs of states
for i in range(num_states):
  for j in range(i + 1, num_states):
    # Check if the geometries of the two states intersect
    if lower48.geometry[i].intersects(lower48.geometry[j]):
        adjacency_matrix[i, j] = 1
        adjacency_matrix[j, i] = 1  # Adjacency matrix is symmetric

Now let's write a rountine that draws the map given a set of 49 input colors

In [ ]:
def draw_map(lower48, color_indices):

  # Define a list of colors
  colors = ['#316879', '#f47a60', '#7fe7dc', '#ced7d8', '#424b4f', '#773c23', '#aec3cf']

  # Assign a random color to each state
  lower48['color'] = [colors[color_indices[i]] for i in range(len(lower48))]

  # Create the plot
  fig, ax = plt.subplots(1, 1, figsize=(10, 6))

  # Plot the states with the assigned colors
  lower48.plot(color=lower48['color'], cmap=None, categorical=True, legend=False, ax=ax, edgecolor='0.8')

  # Customize the plot (optional)
  ax.set_axis_off()

  # Display the plot
  plt.show()

In [ ]:
# Test the drawing routine with random colors
color_indices = np.random.randint(0, 7, size=48)
draw_map(lower48, color_indices)

Now let's write a routine to color the map in such a way that no two adjacent states have the same color

In [ ]:
def exactly_one(x):
  return PbEq([(i,1) for i in x],1)

def graph_coloring(adjacency_matrix, n_colors):
  # Create an n_state by n_color structure (list of lists) of binary variables
  # There will be a 1 in a given position if that state has that particular color
  n_state = adjacency_matrix.shape[0]
  x = [[ z3.Bool("x_{%d,%d}"%((i,j))) for j in range(0,n_colors) ] for i in range(0,n_state) ]
  # Set up the sat solver
  s = Solver()

  # Add constraint one: each state can only have one color
  # For each row of the matrix, exactly one variable is true
  for c_state in range(0,n_state):
    s.add(exactly_one(x[c_state]))

  # Add constraint two.  Two adjacent states cannot have the same color
  # If there is a one in the adjacency matrix at position i,j, then variable
  # x[i, color] and x[j, color] cannot both be true for every color
  for c_state in range(0,n_state):
    for c_neighbor in range(c_state+1,n_state):
      if adjacency_matrix[c_state,c_neighbor]:
        for c_color in range(0,n_colors):
          s.add(Or(Not(x[c_state][c_color]),Not(x[c_neighbor][c_color])))

  # Check if it's SAT (creates the model)
  sat_result = s.check()
  print(sat_result)

  # If it isn't then return
  if sat_result == z3.sat:
    result = s.model()
    x_vals = np.array([[int(bool(result[z3.Bool("x_{%d,%d}" % (i, j))])) for j in range(7)] for i in range(48)])
    color_indices = np.argmax(x_vals, axis=1)
    draw_map(lower48, color_indices)




In [ ]:
# Call the routine to compute the graph coloring with seven colors
graph_coloring(adjacency_matrix, 7)

In [ ]:
# Find the minimum numbers required to color
graph_coloring(adjacency_matrix, 4)
graph_coloring(adjacency_matrix, 3)

# The minimum number is four colors

You can find more about the minimum number of colors required for a planar graph like this in this [Wikipedia page](https://en.wikipedia.org/wiki/Four_color_theorem)